In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import pandas as pd
import numpy as np
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn import svm
## Prep Work
# Load in the data
path ='/content/drive/MyDrive/STAT 4680/STAT 4680 - Team Project!/Prediction Component'
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:

minnesota_data = pd.read_csv('/content/drive/MyDrive/STAT 4680/STAT 4680 - Team Project!/Prediction Component/minnesota_stats_newvar.csv')

minnesota_clusters = pd.read_csv('/content/drive/MyDrive/STAT 4680/STAT 4680 - Team Project!/Prediction Component/player_clusters.csv')
o_clusters = minnesota_clusters[['offensive_cluster']]
d_clusters = minnesota_clusters[['defensive_cluster']]
p_clusters = minnesota_clusters[['playmaking_cluster']]
labels         = minnesota_data.pop('PLAYER_NAME')
print("Labels:", labels)
offensive_data = minnesota_data[['FGM', 'FG3M', 'FTM',  'PTS'  , 'SHOT_DISTANCE', 'FGA', 'FTA', 'FG3A']]
deffensive_data = minnesota_data[['CONTESTED_SHOTS', 'REB', 'STL', 'BLK']]
playmaking_data = minnesota_data[['TOV', 'HEIGHT','WEIGHT', 'AST', 'PF']]
#
print("Cols: ", offensive_data.columns)

#print(minnesota_clusters.head())


Labels: 0       Anthony Edwards
1         Austin Rivers
2           Bryn Forbes
3       Jaden McDaniels
4         Jaylen Nowell
             ...       
122         Ricky Rubio
123    Shabazz Muhammad
124     Tayshaun Prince
125          Tyus Jones
126         Zach LaVine
Name: PLAYER_NAME, Length: 127, dtype: object
Cols:  Index(['FGM', 'FG3M', 'FTM', 'PTS', 'SHOT_DISTANCE', 'FGA', 'FTA', 'FG3A'], dtype='object')


In [3]:

colorado_data = pd.read_csv('/content/drive/MyDrive/STAT 4680/STAT 4680 - Team Project!/Prediction Component/nug_stats_newvar.csv')

colorado_data.dropna(inplace = True, subset=['FGM', 'FG3M', 'FTM',  'PTS'  , 'SHOT_DISTANCE', 'FGA', 'FTA', 'FG3A', 'CONTESTED_SHOTS', 'REB', 'STL', 'BLK', 'TOV', 'HEIGHT','WEIGHT', 'AST', 'PF'])
clabels         = colorado_data.pop('PLAYER_NAME')

seasons = colorado_data.pop('SEASON_ID')
print("Labels:", clabels)
coffensive_data = colorado_data[['FGM', 'FG3M', 'FTM',  'PTS'  , 'SHOT_DISTANCE', 'FGA', 'FTA', 'FG3A']]
print(pd.isna(coffensive_data.isnull().any()))

coffensive_data=coffensive_data.dropna()
print(coffensive_data)
cdeffensive_data = colorado_data[['CONTESTED_SHOTS', 'REB', 'STL', 'BLK']]
cdeffensive_data=cdeffensive_data.dropna()

cplaymaking_data = colorado_data[['TOV', 'HEIGHT','WEIGHT', 'AST', 'PF']]
cplaymaking_data=cplaymaking_data.dropna()

#
print("Cols: ", coffensive_data.columns)
print(coffensive_data.dtypes)


Labels: 0             Aaron Gordon
1              Bruce Brown
2          Christian Braun
3           DeAndre Jordan
4                Ish Smith
              ...         
134         Kenneth Faried
135    Kostas Papanikolaou
136            Mike Miller
137           Nikola Jokic
138            Will Barton
Name: PLAYER_NAME, Length: 138, dtype: object
FGM              False
FG3M             False
FTM              False
PTS              False
SHOT_DISTANCE    False
FGA              False
FTA              False
FG3A             False
dtype: bool
     FGM  FG3M  FTM   PTS  SHOT_DISTANCE   FGA  FTA  FG3A
0    6.3   0.9  2.8  16.3       8.441524  11.2  4.6   2.5
1    4.5   1.1  1.5  11.5      11.735493   9.3  1.9   3.2
2    1.9   0.4  0.5   4.7      10.920415   3.8  0.8   1.3
3    2.3   0.0  0.6   5.1       1.791304   2.9  1.2   0.0
4    1.2   0.0  0.0   2.5      12.916031   3.0  0.1   0.3
..   ...   ...  ...   ...            ...   ...  ...   ...
134  5.2   0.0  2.0  12.5       3.498403   9.3 

In [4]:
print(colorado_data.columns)

Index(['Unnamed: 0', 'PLAYER_ID', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
       'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK',
       'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK',
       'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK',
       'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK',
       'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK',
       'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK',
       'DD2_RANK', 'TD3_RANK', 'WNBA_FANTASY_PTS_RANK', 'SHOT_DISTANCE',
       'HEIGHT', 'WEIGHT', 'NET_RATING', 'CONTESTED_SHOTS', 'OFF_RATING',
       'DEF_RATING', 'PIE', 'USG_PCT', 'POSS', 'DEF_WS', 'POSITION', 'EFF'],
      dtype='object')

In [12]:
# Offensive
o_train, o_test, ocluster_train, ocluster_test = train_test_split(offensive_data, minnesota_clusters['offensive_cluster'], test_size=0.33, random_state=42, shuffle = False)
# Defensive
d_train, d_test, dcluster_train, dcluster_test = train_test_split(deffensive_data, minnesota_clusters['defensive_cluster'], test_size=0.33, random_state=42, shuffle = False)
# Playmaking
p_train, p_test, pcluster_train, pcluster_test = train_test_split(playmaking_data, minnesota_clusters['playmaking_cluster'], test_size=0.33, random_state=42, shuffle = False)


print(coffensive_data.dtypes)
print(o_test.columns)

FGM              float64
FG3M             float64
FTM              float64
PTS              float64
SHOT_DISTANCE    float64
FGA              float64
FTA              float64
FG3A             float64
dtype: object
Index(['FGM', 'FG3M', 'FTM', 'PTS', 'SHOT_DISTANCE', 'FGA', 'FTA', 'FG3A'], dtype='object')


In [15]:

# Offensive
clf = RandomForestClassifier( random_state=5)
clf.fit(o_train, ocluster_train)
print(clf.predict(o_test))
print(list(ocluster_test))

predicted = clf.predict(o_test)
test_clusters = list(ocluster_test)
count = 0
# Testing Difference
for cluster in range(len(predicted)):
  if(predicted[cluster] == test_clusters[cluster]):
    count = count +1

print(count/len(predicted))
co_predicted_rf = clf.predict(coffensive_data)

# clf = svm.SVC()
# clf.fit(o_train, ocluster_train)
# predicted = clf.predict(o_test)
# test_clusters = list(ocluster_test)
# count = 0
# # Testing Difference
# for cluster in range(len(predicted)):
#   if(predicted[cluster] == test_clusters[cluster]):
#     count = count +1

# print(count/len(predicted))
# co_predicted_svm  = clf.predict(coffensive_data)
# Random forest perfectly predicted our clustering method which means it should extend to other data



[5 3 3 5 4 4 1 3 1 5 5 4 5 5 1 5 3 4 5 3 1 5 5 3 4 4 5 2 5 1 5 4 3 4 5 5 3
 4 4 5 5 4]
[5, 3, 3, 5, 4, 4, 1, 3, 1, 5, 5, 4, 5, 5, 1, 5, 3, 4, 5, 3, 1, 5, 5, 3, 4, 4, 5, 2, 5, 1, 5, 4, 3, 4, 5, 5, 3, 4, 4, 5, 5, 4]
1.0


In [22]:
from sklearn.tree import export_graphviz
# Export as dot file
tree = clf.estimators_[2]
export_graphviz(tree, out_file='tree.dot',
                feature_names = o_test.columns,
                class_names = ["Primary Scorer","Long Range Shooters","Poor Scorers","Average Scorers","Poor Long Range Shooters"],
                rounded = True, proportion = False,
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

Output hidden; open in https://colab.research.google.com to view.

In [7]:
#Defensive
clf = RandomForestClassifier( random_state=5)
clf.fit(d_train, dcluster_train)
print(clf.predict(d_test))
print(list(dcluster_test))

predicted = clf.predict(d_test)
test_clusters = list(dcluster_test)
count = 0
# Testing Difference
for cluster in range(len(predicted)):
  if(predicted[cluster] == test_clusters[cluster]):
    count = count +1

print(count/len(predicted))

# Random forest predicted our clustering method well which means it should extend to other data
cd_predicted_rf = clf.predict(cdeffensive_data)
print(cd_predicted_rf)

clf = svm.SVC()
clf.fit(d_train, dcluster_train)
print(clf.predict(d_test))
print(list(dcluster_test))

predicted = clf.predict(d_test)
test_clusters = list(dcluster_test)
count = 0
# Testing Difference
for cluster in range(len(predicted)):
  if(predicted[cluster] == test_clusters[cluster]):
    count = count +1

print(count/len(predicted))

# Random forest predicted our clustering method well which means it should extend to other data
cd_predicted_rf = clf.predict(cdeffensive_data)
cd_predicted_svm = clf.predict(cdeffensive_data)
print(cd_predicted_rf)





[2 1 1 4 1 5 5 1 3 1 4 4 4 1 4 1 1 4 1 1 3 4 4 4 5 1 1 4 1 4 1 4 1 3 4 1 1
 5 1 1 1 4]
[2, 1, 1, 4, 1, 5, 5, 1, 3, 1, 4, 4, 4, 1, 4, 1, 1, 3, 1, 1, 3, 4, 4, 4, 5, 1, 1, 4, 1, 4, 1, 3, 1, 3, 4, 1, 1, 5, 1, 1, 1, 4]
0.9523809523809523
[4 4 1 4 1 1 4 1 5 4 5 1 4 4 1 1 4 4 1 1 1 1 4 4 4 1 1 1 4 4 5 4 1 1 4 1 4
 4 1 4 1 1 4 4 1 4 4 5 4 4 1 1 4 1 4 5 4 4 1 4 4 4 4 1 1 4 1 1 1 1 4 1 1 4
 1 4 1 1 4 4 4 1 5 4 1 4 4 1 1 1 1 5 4 1 1 1 4 2 3 4 1 1 4 4 4 1 4 1 4 4 1
 4 1 1 4 1 4 1 3 4 4 4 1 1 4 4 4 1 4 1 1 1 4 4 1 1 4 4]
[1 1 1 4 1 4 4 1 3 1 4 4 1 1 4 1 1 4 1 1 3 4 4 1 4 1 1 4 1 4 1 4 1 3 4 4 1
 4 4 1 1 4]
[2, 1, 1, 4, 1, 5, 5, 1, 3, 1, 4, 4, 4, 1, 4, 1, 1, 3, 1, 1, 3, 4, 4, 4, 5, 1, 1, 4, 1, 4, 1, 3, 1, 3, 4, 1, 1, 5, 1, 1, 1, 4]
0.7380952380952381
[4 4 1 4 1 1 4 1 4 4 3 1 1 4 1 1 4 1 1 1 1 1 4 1 4 1 4 1 4 4 3 4 1 1 4 4 4
 1 1 4 1 4 4 4 1 4 1 3 4 4 1 1 4 1 1 4 4 4 4 4 4 1 3 4 1 4 4 1 1 1 4 1 1 4
 1 4 1 4 1 4 1 1 3 4 1 4 4 1 4 1 1 4 4 1 4 1 4 1 3 4 1 4 4 4 4 1 4 1 4 4 1
 1 1 4 4 1 4 1 3 4 4 4 1 1 4

In [8]:
# Playmaking
clf = RandomForestClassifier( random_state=5)
clf.fit(p_train, pcluster_train)
print(clf.predict(p_test))
print(list(pcluster_test))

predicted = clf.predict(p_test)
test_clusters = list(pcluster_test)
count = 0
# Testing Difference
for cluster in range(len(predicted)):
  if(predicted[cluster] == test_clusters[cluster]):
    count = count +1

print(count/len(predicted))
cp_predicted_rf = clf.predict(cplaymaking_data)
# Random forest predicted our clustering method well which means it should extend to other data


# Playmaking
clf = svm.SVC()
clf.fit(p_train, pcluster_train)
print(clf.predict(p_test))
print(list(pcluster_test))

predicted = clf.predict(p_test)
test_clusters = list(pcluster_test)
count = 0
# Testing Difference
for cluster in range(len(predicted)):
  if(predicted[cluster] == test_clusters[cluster]):
    count = count +1

print(count/len(predicted))
cp_predicted_svm = clf.predict(cplaymaking_data)
# Random forest predicted our clustering method well which means it should extend to other data


[5 3 4 3 4 1 6 3 6 5 3 3 4 3 2 5 3 6 4 3 6 2 3 3 1 5 4 2 3 2 5 6 3 6 3 3 3
 1 5 5 4 2]
[5, 3, 4, 3, 4, 1, 2, 3, 6, 5, 3, 3, 4, 3, 2, 5, 3, 6, 4, 3, 6, 2, 3, 3, 1, 5, 4, 2, 3, 2, 5, 6, 3, 6, 3, 3, 3, 1, 5, 5, 4, 2]
0.9761904761904762
[5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 3
 5 5 5 5 5]
[5, 3, 4, 3, 4, 1, 2, 3, 6, 5, 3, 3, 4, 3, 2, 5, 3, 6, 4, 3, 6, 2, 3, 3, 1, 5, 4, 2, 3, 2, 5, 6, 3, 6, 3, 3, 3, 1, 5, 5, 4, 2]
0.19047619047619047


In [9]:

data = {'Name': clabels,
        'Season': seasons
        ,
        'Offensive Cluster RF': co_predicted_rf,
        'Offensive Cluster SVM': co_predicted_svm ,
        'Defensive Cluster RF': cd_predicted_rf,
        'Defensive Cluster SVM': cd_predicted_svm,
        'Playmaking Cluster RF': cp_predicted_rf,
        'Playmaking Cluster SVM': cp_predicted_svm
        }

print(data)
pd.DataFrame(data).to_csv('/content/drive/MyDrive/STAT 4680/STAT 4680 - Team Project!/Prediction Component/nug_clusters.csv')

{'Name': 0             Aaron Gordon
1              Bruce Brown
2          Christian Braun
3           DeAndre Jordan
4                Ish Smith
              ...         
134         Kenneth Faried
135    Kostas Papanikolaou
136            Mike Miller
137           Nikola Jokic
138            Will Barton
Name: PLAYER_NAME, Length: 138, dtype: object, 'Season': 0      2022-23
1      2022-23
2      2022-23
3      2022-23
4      2022-23
        ...   
134    2015-16
135    2015-16
136    2015-16
137    2015-16
138    2015-16
Name: SEASON_ID, Length: 138, dtype: object, 'Offensive Cluster RF': array([4, 4, 5, 3, 5, 5, 2, 4, 4, 2, 4, 5, 4, 4, 5, 3, 4, 5, 4, 4, 5, 5,
       4, 5, 5, 5, 4, 5, 4, 4, 1, 5, 5, 5, 2, 5, 4, 4, 5, 5, 5, 4, 3, 2,
       5, 2, 4, 1, 4, 4, 5, 5, 4, 5, 5, 4, 2, 4, 3, 3, 4, 4, 4, 3, 5, 4,
       5, 5, 3, 5, 4, 5, 5, 4, 4, 2, 3, 5, 4, 3, 4, 5, 4, 4, 5, 5, 4, 5,
       4, 5, 4, 2, 2, 5, 3, 5, 3, 3, 1, 4, 5, 5, 4, 1, 4, 5, 1, 5, 4, 4,
       4, 4, 3, 5, 4, 5, 4, 5, 4, 3, 4